## Predicting Heart Disease Using Machine Learning

We attempt to create a machine learning model that detects heart disease based on the medical records of patients.  
This notebook uses various Python libraries for data science and machine learning.

## Approach

1. Problem definition  
2. Data  
3. Evaluation  
4. Features  
5. Modeling  
6. Experimentation

## Problem definition

Given certain medical records of a patient, is it possible to detect whether the patient suffers from heart disease?  
The machine learning problem is **supervised learning / binary classification**.

## Data

The data we use is the Cleveland Heart Disease Dataset, which is publicly available:  
[UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/45/heart+disease)  
[Kaggle](https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data)

## Evaluation

We are trying to achieve 95% accuracy with the proof of concept model to pursue the project further.

## Features

**Data Dictionary (information about each data feature)**  
* age (age of patient in years)  
* sex (0=female, 1=male)  
* cp (chest pain type: 1=typical angina, 2=atypical angina, 3=non-anginal, 4=asymptomatic)  
* trestbps (resting blood pressure in mmHg on admission to hospital)  
* chol (serum cholesterol in mg/dl)  
* fbs (whether fasting blood sugar is > 120 mg/dl: 0=no, 1=yes)  
* restecg (resting electrocardiographic results: 0=normal, 1=ST/T abnormality, 2=left ventricular hypertrophy)  
* thalach (maximum heart rate achieved)  
* exang (exercise-induced angina: 0=no, 1=yes)  
* oldpeak (ST segment depression induced by exercise relative to resting)  
* slope (slope of the peak exercise ST segment: 1=upsloping, 2=flat, 3=downsloping)  
* ca (number of major vessels [0-3] colored by fluoroscopy)  
* thal (1-3=normal, 6=fixed defect, 7=reversible defect)  
* target (the predicted attribute: 0=no heart disease, 1=heart disease)

## Preparing the Tools

Python libraries Numpy, Pandas, and Matplotlib are used for data analysis and manipulation.  
Python library Scikit-Learn (SkLearn) is used for machine learning.

In [ ]:
### importing exploratory data analysis (EDA) tools
import numpy, pandas, seaborn
from matplotlib import pyplot

### rendering plots inside this notebook
%matplotlib inline

### importing sklearn model selection tools
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV

### importing sklearn machine learning algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

### importing sklearn model evaluation tools
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import RocCurveDisplay

## Loading Data

In [ ]:
### importing from csv file
heart_disease = pandas.read_csv(filepath_or_buffer="data-heart-disease.csv")

In [ ]:
### displaying dimensins of the dataframe
heart_disease.shape

In [ ]:
### displaying the first 5 rows
heart_disease.head()

In [ ]:
### displaying the last 5 rows
heart_disease.tail()

## Exploratory Data Analysis = EDA

The goal is to become a subject matter expert on the dataset.  
1. Data basics: data types, numeric/categorical, statistics, balanced/imbalanced, etc...
2. Cleaning data: handling missing values and outliers
3. Transforming data: common units, standardization, encoding, etc...
4. Data engineering: creating new features from existing ones
5. Reducing data: removing non-relevant features

In [ ]:
### displaying dataframe basic information
heart_disease.info()

In [ ]:
### displaying basic statistics
heart_disease.describe()

In [ ]:
### counting category instances of the target variable
heart_disease["target"].value_counts()

In [ ]:
### visualizing category instances of the target variable
heart_disease["target"].value_counts().plot(kind="bar", color=["salmon", "lightblue"])
pyplot.title(label="Category Instances of Target Variable")
pyplot.ylabel(ylabel="Counts")
pyplot.xlabel(xlabel="Target Variable (Heart Disease): 0=No 1=Yes")
pyplot.xticks(rotation=0);

In [ ]:
### checking for missing values
heart_disease.isna().sum()

#### Sex of patient and heart disease

In [ ]:
### counting category instances of sex variable :)
### sex: 0=female, 1=male
heart_disease["sex"].value_counts()

In [ ]:
### comparing sex column to target column with cross tabulation
### sex: 0=female, 1=male
### target: 0=no disease, 1=disease
pandas.crosstab(index=heart_disease["sex"], columns=heart_disease["target"])

In [ ]:
### plotting heart disease frequency over sex of patient
pandas.crosstab(index=heart_disease["sex"], columns=heart_disease["target"]).plot(
    kind="bar", color=["lightblue", "salmon"])
pyplot.title(label="Heart Disease Frequency by Sex")
pyplot.ylabel(ylabel="Patient Count")
pyplot.xlabel(xlabel="Sex of Patient")
pyplot.legend(["No Heart Disease", "Heart Disease"])
pyplot.xticks(ticks=[0,1], labels=["Female", "Male"], rotation=0);

#### Maximum Heart Rate vs. Age and Heart Disease

In [ ]:
### histogram: distribution of maximum heart rate values
heart_disease["thalach"].plot.hist();

In [ ]:
### histogram: distribution of age values
heart_disease["age"].plot.hist();

In [ ]:
### creating figure object
pyplot.figure(figsize=(10,6))

### scatter plot: maximum heart rate over age in no heart disease
pyplot.scatter(
    y=heart_disease["thalach"][heart_disease["target"]==0],
    x=heart_disease["age"][heart_disease["target"]==0],
    color="lightblue")

### scatter plot: maximum heart rate over age in heart disease
pyplot.scatter(
    y=heart_disease["thalach"][heart_disease["target"]==1],
    x=heart_disease["age"][heart_disease["target"]==1],
    color="salmon")

### configuring scatter plots
pyplot.title(label="Maximum Heart Rate vs. Age and Heart Disease")
pyplot.legend(["No Heart Disease", "Heart Disease"]);
pyplot.ylabel(ylabel="Maximum Heart Rate")
pyplot.xlabel(xlabel="Age of Patient");

#### Chest pain type and heart disease

*cp (chest pain type)*
* 1 = typical angina (chest pain related to impaired blood supply of heart)
* 2 = atypical angina (chest pain not related to impaired blood supply of heart)
* 3 = non-anginal (typically easophageal spasm)
* 4 = asymptomatic (chest pain not related to heart disease)

In [ ]:
### crosstab: chest pain type vs heart disease
pandas.crosstab(index=heart_disease["cp"], columns=heart_disease["target"])

In [ ]:
### bar graph: heart disease frequency over chest pain type
pandas.crosstab(index=heart_disease["cp"], columns=heart_disease["target"]).plot(
    kind="bar", color=["lightblue", "salmon"])

### configuring bar graph
pyplot.title(label="Heart Disease Frequency Over Chest Pain Type")
pyplot.legend(["No Heart Disease", "Heart Disease"])
pyplot.ylabel(ylabel="Patient Count")
pyplot.xlabel(xlabel="Chest Pain Type")
pyplot.xticks(ticks=[0,1,2,3], labels=["Typical Angina", "Atypical Angina", "Non-Anginal", "Asymptomatic"], rotation=0);

#### Correlation matrix

In [ ]:
### creating correlation matrix with pandas
correlation_matrix = heart_disease.corr()
correlation_matrix

In [ ]:
### visualizing correlation matrix with seaborn
figure, axis = pyplot.subplots(figsize=(14,10))
axis = seaborn.heatmap(data=correlation_matrix, linewidths=1.0, cmap="YlGnBu", annot=True, fmt=".2f")

## Modeling

#### Preparing data

Splitting data into training and test sets.  
The machine learning algorithm is trained (finds patterns) on the training set.  
The trained algorithm is then tested (applies patterns) on the test set.

In [ ]:
### splitting data features <> target
features = heart_disease.drop(columns="target")
target = heart_disease.loc[:, "target"]

### splitting data train <> test
numpy.random.seed(seed=42)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)

#### Selecting machine learning algorithm

**Three machine learning algorithms will be applied and compared**

1. Logistic Regression
2. K-Nearest Neighbours Classifier
3. Random Forest Classifier

These algorithms were selected from to the sklearn model selection map.  
[Sklearn Model Selection Map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

In [ ]:
### creating algorithm dictionary
algo_dict = {
    "Logistic Regression": LogisticRegression(),
    "K-Nearest Neighbours Classifier": KNeighborsClassifier(),
    "Random Forest Classifier": RandomForestClassifier()}

### creating scores dictionary
score_dict = {}

### training and evaluating algorithms
numpy.random.seed(seed=42)
for name, algo in algo_dict.items():
    algo.fit(features_train, target_train)
    score_dict[name] = algo.score(features_test, target_test)

### displaying scores
score_dict

#### Comparing algorithms

In [ ]:
### visualizing algorithm performance
score_df = pandas.DataFrame(data=score_dict, index=["Accuracy"])
score_df.T.plot(kind="bar", figsize=(8,6), legend=None)
pyplot.title(label="Algorithm Performance on the Heart Disease Dataset", fontsize=14)
pyplot.ylabel(ylabel="Accuracy", fontsize=14)
pyplot.xticks(rotation=0)
pyplot.xlabel(xlabel="Machine Learning Algorithm", fontsize=14);

#### Model improvement / Hyperparameter tuning

We have a baseline model that we try to improve using:
* hyperparameter tuning
* feature selection

Tools for evaluating the performance of a classification algorithm:
* confusion matrix
* classification report
* accuracy / precision / recall / f1-score
* cross-validation
* ROC curve / area under the ROC curve

In [ ]:
### nearest neighbors classifier hyperparameter tuning by hand
train_scores = []
test_scores = []
knn_classifier = KNeighborsClassifier()
for k_neighbors in range(1,21):
    knn_classifier.set_params(n_neighbors=k_neighbors)
    knn_classifier.fit(X=features_train, y=target_train)
    train_scores.append(knn_classifier.score(X=features_train, y=target_train))
    test_scores.append(knn_classifier.score(X=features_test, y=target_test))

In [ ]:
### plotting nearest neighbors classifier tuning scores
pyplot.plot(range(1,21), train_scores, label="Training Scores")
pyplot.plot(range(1,21), test_scores, label="Test Scores")
pyplot.title(label="Nearest Neighbors Classifier Hyperparameter Tuning")
pyplot.ylabel(ylabel="Accurarcy")
max_score = max(test_scores)
pyplot.axhline(y=max_score, color="black", linestyle="dotted", label=f"Best Score = {max_score:.3f}")
pyplot.xlabel(xlabel="Number of Neighbors")
pyplot.xticks(ticks=range(1,21,2))
best_param = test_scores.index(max(test_scores)) + 1
pyplot.axvline(x=best_param, color="black", linestyle="dashed", label=f"Best Param = {best_param}")
pyplot.legend();

#### Hyperparameter tuning using Randomized Search Cross Validation

Performance of the K Nearest Neighbors Classifier algorithm is too weak - no further use...  
We continue by searching for better hyperparameters of Logistic Regression and Random Forest Classifier.

In [ ]:
### logistic regression: creating hyperparameter grid
logreg_grid = {
    "C": numpy.logspace(start=-4, stop=4, num=20),
    "solver": ["liblinear"]}

In [ ]:
### logistic regression: randomized search for best parameters
numpy.random.seed(seed=42)
logreg_random = RandomizedSearchCV(
    estimator=LogisticRegression(), param_distributions=logreg_grid, cv=5, n_iter=20, verbose=True)
logreg_random.fit(X=features, y=target)
logreg_random.best_params_

In [ ]:
### logistic regression: scoring best model
logreg_random.score(X=features_test, y=target_test)

In [ ]:
### random forest classifier: creating hyperparameter grid
forest_grid = {
    "n_estimators": range(10, 1000, 50),
    "max_depth": [None, 3, 5, 10],
    "min_samples_split": range(2, 20, 2),
    "min_samples_leaf": range(1, 20, 2)}

In [ ]:
### random forest classifier: randomized search for best parameters
numpy.random.seed(seed=42)
forest_random = RandomizedSearchCV(
    estimator=RandomForestClassifier(), param_distributions=forest_grid, cv=5, n_iter=20, verbose=True)
forest_random.fit(X=features, y=target)
forest_random.best_params_

In [ ]:
### random forest classifier: scoring best model
forest_random.score(X=features_test, y=target_test)

#### Hyperparameter tuning using Grid Search Cross Validation

After randomized hyperparameter search, the Logistic Regression algorithm provides the best performance.  
We continue by tuning the Logistic Regression algorithm even further (if possible).  
We use Grid Search Cross Validation that performs an exhaustive hyperparameter search.

In [ ]:
### logistic regression: creating hyperparameter dictionary
logreg_params = {
    "C": numpy.logspace(start=-4, stop=4, num=30),
    "solver": ["liblinear"]}

In [ ]:
### logistic regression: grid search for best parameters
logreg_grid = GridSearchCV(estimator=LogisticRegression(), param_grid=logreg_params, cv=5, verbose=True)
logreg_grid.fit(X=features, y=target)
logreg_grid.best_params_

In [ ]:
### logistic regression: scoring best model
logreg_grid.score(X=features_test, y=target_test)

#### Model evaluation

**The following metrics will be computed:**
* Confusion Matrix
* Classification Report
* Precision
* Recall
* F1 Score
* ROC Curve and AUC Score

Cross validation will be applied where possible.

In [ ]:
### making predictions with best model
target_preds = logreg_grid.predict(X=features_test)

In [ ]:
### plotting roc curve of best model
RocCurveDisplay.from_estimator(estimator=logreg_grid, X=features_test, y=target_test);

In [ ]:
### plotting confusion matrix using seaborn heatmap
seaborn.set(font_scale=1.5)
figure, axis = pyplot.subplots(figsize=(3,3))
axis = seaborn.heatmap(data=confusion_matrix(y_true=target_test, y_pred=target_preds), annot=True, cbar=False)
axis.set_ylabel(ylabel="Target Labels")
axis.xaxis.set_ticks_position(position="top")
axis.tick_params(axis="x", which="both", top=False)
axis.xaxis.set_label_position(position="top")
axis.set_xlabel(xlabel="Predicted Labels", labelpad=10);

In [ ]:
### obtaining classification report
print(classification_report(y_true=target_test, y_pred=target_preds))